In [36]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import AnimalShelterClient as client





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "12345"
shelter = client.AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Austin Animal Center (AAC) Dashboard'))),
    html.P("Page made by Brandon Thibeaux"),
    html.Hr(),
    # customer image location, can you put it in different location and resize?
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div( 
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
            dcc.Dropdown(id='dropdown',
                 options=[
                     {'label': 'Water Rescue', 'value': 'Water'},
                     {'label': 'Mountain or Wilderness Rescue', 'value': 'Wilderness'},
                     {'label': 'Disaster or Individual Tracking Rescue', 'value': 'Disaster'},
                     {'label': 'Reset', 'value': 'All'},],
                 value='All',multi=False,clearable=False),
    ),

    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
    #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    #If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Filtering options
@app.callback(
    Output('datatable-id', "data"),
    [Input('dropdown', 'value')]
)
def update_output(value):
    data = pd.DataFrame.from_records(shelter.readAll({}))

    if(value == 'Water'):
        data = pd.DataFrame(list(shelter.readAll(
            {'breed': { '$in': ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             'sex_upon_outcome':"Intact Female",
            'age_upon_outcome_in_weeks':{'$gt' :  26, '$lt': 156}})))
        
    elif(value == 'Wilderness'):
        data = pd.DataFrame(list(shelter.readAll(
            {
                'breed': { '$in': ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
                'sex_upon_outcome':"Intact Male",
                'age_upon_outcome_in_weeks':{'$gte' :  26, '$lte': 156}
            })))
    elif(value == 'Disaster'):
        data = pd.DataFrame(list(shelter.readAll(
            {
                'breed': { '$in': ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
                'sex_upon_outcome':"Intact Male",
                'age_upon_outcome_in_weeks':{'$gte' :  20, '$lte': 300}
            })))
    elif(value == 'All'):
         data = pd.DataFrame.from_records(shelter.readAll({}))
    else: # Here for security
        data = pd.DataFrame.from_records(shelter.readAll({}))

    return data.to_dict('records')

# UPDATES AS USER SELECTS A CELL
'''
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

'''


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Updates Graph
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)

    # add code for chart of your choice (e.g. pie chart) #
    return [
        dcc.Graph(            
            figure = px.pie(data_frame=dff ,names=dff['breed'])
        )    
    ]
    
    
    '''
    NOTES:
        ViewData contains column names
        dff= contains a copy of the data frame
    '''


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    #FIXME: Add in the code for your geolocation chart
    #If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    rowIndex = 0; # looks the row or document on the table
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[rowIndex,13],dff.iloc[rowIndex,14]], children=[
                dl.Tooltip(dff.iloc[rowIndex,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[rowIndex,9])
                ])
            ])
        ])
    ]


app